# Q1. You are working on a machine learning project where you have a dataset containing numerical and categorical features. You have identified that some of the features are highly correlated and there are missing values in some of the columns. You want to build a pipeline that automates the feature engineering process and handles the missing values.

## Design a pipeline that includes the following steps:
* __Use an automated feature selection method to identify the important features in the dataset.__
* __Create a numerical pipeline that includes the following steps.__
* __Impute the missing values in the numerical columns using the mean of the column values.__
* __Scale the numerical columns using standardisation.__
* __Create a categorical pipeline that includes the following steps.__
* __Impute the missing values in the categorical columns using the most frequent value of the column.__
* __One-hot encode the categorical columns.__
* __Combine the numerical and categorical pipelines using a ColumnTransformer.__
* __Use a Random Forest Classifier to build the final model.__
* __Evaluate the accuracy of the model on the test dataset.__

__Note__: Your solution should include code snippets for each step of the pipeline, and a brief explanation of
each step. You should also provide an interpretation of the results and suggest possible improvements for
the pipeline.

In [ ]:
# import necessary libraries
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import seaborn as sns
# load the data
df = sns.load_dataset('tips')

#hot label encoding for Time feature
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

## independent and dependent features
X=df.drop(labels=['time'],axis=1)
y=df['time']

# split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create a feature selection pipeline using SelectKBest and f_regression
feature_selection_pipeline = Pipeline([
    ('selector', SelectKBest(score_func=f_regression)),
])

# create a numerical pipeline using SimpleImputer and StandardScaler
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
])

# create a categorical pipeline using SimpleImputer and OneHotEncoder
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore')),
])

# combine the numerical and categorical pipelines using ColumnTransformer
preprocessor = ColumnTransformer([
    ('num', numerical_pipeline, X_train.select_dtypes(include=['float64', 'int64']).columns),
    ('cat', categorical_pipeline, X_train.select_dtypes(include=['object']).columns)
])

# create the final pipeline by combining the feature selection and preprocessor pipelines with a RandomForestClassifier
pipeline = Pipeline([
    ('feat_select', feature_selection_pipeline),
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

# fit the pipeline to the training data
pipeline.fit(X_train, y_train)

# evaluate the pipeline on the test data
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In this code, X and y represent the dataset and the target variable, respectively. num_features and cat_features are the lists of the names of the numerical and categorical features, respectively. The code first splits the dataset into training and test sets. Then, it creates the numerical and categorical pipelines, which impute missing values and scale/encode the data, respectively. Next, it combines the pipelines using a ColumnTransformer. Finally, it builds the final pipeline with the preprocessor and a RandomForestClassifier as the classifier. The pipeline is fit on the training data and evaluated on the test data using the accuracy score.

#  Q2. Build a pipeline that includes a random forest classifier and a logistic regression classifier, and then use a voting classifuir to combine their predictions. Train the pipeline on the iris dataset and evaluate its accuracy.

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Load the iris dataset and split it into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the individual classifiers
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
lr = LogisticRegression(random_state=42)

# Create the voting classifier
voting_clf = VotingClassifier(
    estimators=[('rfc', rfc), ('lr', lr)],
    voting='hard'
)

# Create the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('voting', voting_clf)
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Evaluate the accuracy of the model on the test data
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

In this code, X and y represent the dataset and the target variable, respectively. The code first splits the dataset into training and test sets. Then, it creates the individual classifiers: a RandomForestClassifier and a LogisticRegression classifier. Next, it creates a VotingClassifier that combines the two classifiers using hard voting. Finally, it creates a pipeline that scales the data and applies the VotingClassifier. The pipeline is fit on the training data and evaluated on the test data using the accuracy score.